In [ ]:
# | default_exp ensemble
# | default_cls_lvl 3

In [ ]:
#| hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# ensemble

> TODO: Add information about this particular model

In [ ]:
#| export
import os, gc, time
from functools import partial
from pathlib import Path

import glob
import numpy as np
import pandas as pd
from scipy.optimize import fmin, minimize
import torch

from kaggle_comp import utils

In [ ]:
# | hide_input
import pdb
from fastcore.test import *

## Get OOF files

In [ ]:
*_, model_output_path, log_output_path = utils.get_paths()
log_output_path

Path('../logs')

In [ ]:
all_oof_files = glob.glob(f"{str(log_output_path)}/oof_*.csv")
all_oof_files[:5]

['../logs/oof_blurr_deberta_v3_small_2022_11_23_17_53_4_fold_4.csv',
 '../logs/oof_blurr_deberta_v3_base_2022_11_23_19_53_2_fold_2.csv',
 '../logs/oof_blurr_deberta_v3_base_2022_11_21_23_50_2_fold_2.csv',
 '../logs/oof_blurr_deberta_v3_large_2022_11_23_18_58_4_fold_4.csv',
 '../logs/oof_blurr_deberta_v3_base_2022_11_21_23_04_1_fold_1.csv']

In [ ]:
use_models = [
    'oof_blurr_cocolm_large_2022_11_28_14_58', 
    'oof_blurr_deberta_v3_base_2022_11_23_16_10', 
    # 'oof_blurr_deberta_v3_large_2022_11_23_16_18', 
    # 'oof_blurr_deberta_xlarge_2022_11_25_20_54',
    # 'oof_blurr_deberta_large_2022_11_25_20_33'
] 

oof_files = sorted([f for f in all_oof_files if Path(f).stem.startswith(tuple(use_models))])
oof_files[:5]

['../logs/oof_blurr_cocolm_large_2022_11_28_14_58_0_fold_0.csv',
 '../logs/oof_blurr_cocolm_large_2022_11_28_14_58_1_fold_1.csv',
 '../logs/oof_blurr_cocolm_large_2022_11_28_14_58_2_fold_2.csv',
 '../logs/oof_blurr_cocolm_large_2022_11_28_14_58_3_fold_3.csv',
 '../logs/oof_blurr_cocolm_large_2022_11_28_14_58_4_fold_4.csv']

## Merge OOF model results

In [ ]:
x = torch.tensor([1.7001953,1.5380859,2.0488281,1.8027344,1.7763672,1.6572266])
x[None,:].shape

torch.Size([1, 6])

In [ ]:
loss_func = utils.MCRMSELoss()
loss_func(torch.tensor([1.7001953,1.5380859,2.0488281,1.8027344,1.7763672,1.6572266])[None,:], torch.tensor([2.0,2.0,2.0,2.5,2.0,2.0])[None,:]).item()

0.345706045627594

In [ ]:

loss_func = utils.MCRMSELoss()

def calc_example_metric(r):
    loss = loss_func(
        torch.tensor([r["pred_cohesion"],r["pred_syntax"],r["pred_vocabulary"],r["pred_phraseology"],r["pred_grammar"],r["pred_conventions"]])[None,:], 
        torch.tensor([r["targ_cohesion"],r["targ_syntax"],r["targ_vocabulary"],r["targ_phraseology"],r["targ_grammar"],r["targ_conventions"]])[None,:]
    )
    
    return loss.item()

In [ ]:
Path(oof_files[0]).stem[:-9]

'oof_blurr_cocolm_large_2022_11_28_14_58'

In [ ]:
df = None
run_ids = set()
for f in oof_files:
    fold = Path(f).stem[-1]
    
    tmp_df = pd.read_csv(f)
    tmp_df.insert(0, 'id', range(len(tmp_df)))
    tmp_df.insert(1, 'fold', int(fold))
    tmp_df["id"] = tmp_df["id"].apply(lambda v: f"fold_{fold}_{str(v)}")
    
    run_id = Path(f).stem[:-9] #tmp_df.iloc[0]["run_id"]
    run_ids.add(run_id)

    # calculate the metric for each row
    tmp_df[f"{run_id}_loss"] = tmp_df.apply(calc_example_metric, axis=1)

    # # move targets to end
    # cols = list(tmp_df.columns.values)
    # cols.pop(cols.index('Adequate')) 
    # cols.pop(cols.index('Effective')) 
    # cols.pop(cols.index('Ineffective')) 
    # tmp_df = tmp_df[cols + ['Adequate','Effective', 'Ineffective']] 

    # add run_id as a suffix to target label to distringuish one run from the next
    tmp_df.rename(columns={
        "pred_cohesion": f"pred_cohesion_{run_id}", 
        "pred_syntax": f"pred_syntax_{run_id}", 
        "pred_vocabulary": f"pred_vocabulary_{run_id}",
        "pred_phraseology": f"pred_phraseology_{run_id}", 
        "pred_grammar": f"pred_grammar_{run_id}", 
        "pred_conventions": f"pred_conventions_{run_id}",
    }, inplace=True)
    
    tmp_df.drop(columns=["run_id"], inplace=True)

    if df is None:
        df = tmp_df
    else:
        df = df.merge(tmp_df, on="id", how="left", suffixes=('', '_drop'))
        df.drop([col for col in df.columns if 'drop' in col], axis=1, inplace=True)

In [ ]:
df.head()

,id,fold,pred_cohesion_oof_blurr_cocolm_large_2022_11_28_14_58,pred_syntax_oof_blurr_cocolm_large_2022_11_28_14_58,pred_vocabulary_oof_blurr_cocolm_large_2022_11_28_14_58,pred_phraseology_oof_blurr_cocolm_large_2022_11_28_14_58,pred_grammar_oof_blurr_cocolm_large_2022_11_28_14_58,pred_conventions_oof_blurr_cocolm_large_2022_11_28_14_58,targ_cohesion,targ_syntax,...,targ_conventions,grid_id,oof_blurr_cocolm_large_2022_11_28_14_58_loss,pred_cohesion_oof_blurr_deberta_v3_base_2022_11_23_16_10,pred_syntax_oof_blurr_deberta_v3_base_2022_11_23_16_10,pred_vocabulary_oof_blurr_deberta_v3_base_2022_11_23_16_10,pred_phraseology_oof_blurr_deberta_v3_base_2022_11_23_16_10,pred_grammar_oof_blurr_deberta_v3_base_2022_11_23_16_10,pred_conventions_oof_blurr_deberta_v3_base_2022_11_23_16_10,oof_blurr_deberta_v3_base_2022_11_23_16_10_loss
0,fold_0_0,0,3.162109,3.275391,3.298828,3.296875,3.458984,3.083984,2.5,2.5,...,3.0,0,0.596030,1.724609,1.588867,2.007812,1.796875,1.785156,1.735352,0.312837
1,fold_0_1,0,3.542969,3.664062,3.634766,3.691406,3.724609,3.429688,3.0,3.0,...,3.0,0,0.531251,2.925781,2.685547,3.074219,2.960938,2.923828,2.882812,0.449223
2,fold_0_2,0,3.638672,3.730469,3.662109,3.708984,3.638672,3.718750,4.5,4.0,...,4.0,0,0.446616,3.726562,3.644531,3.869141,3.925781,3.929688,3.835938,0.301761
3,fold_0_3,0,3.560547,3.529297,3.509766,3.380859,3.316406,3.742188,2.0,3.0,...,2.5,0,1.089844,3.478516,3.207031,3.427734,3.345703,2.994141,3.228516,0.385096
4,fold_0_4,0,3.306641,3.371094,3.410156,3.318359,3.332031,3.419922,4.0,4.0,...,3.5,0,0.417645,3.318359,3.027344,3.302734,3.158203,2.968750,3.304688,0.440434


In [ ]:
run_ids

{'oof_blurr_cocolm_large_2022_11_28_14_58',
 'oof_blurr_deberta_v3_base_2022_11_23_16_10'}

In [ ]:
print('Avg. overall loss')
loss_cols = [run_id+'_loss' for run_id in run_ids]
avg_loss = np.mean(df[loss_cols].values, axis=0)
print(np.mean(avg_loss))

Avg. overall loss
0.37886852147700767


In [ ]:
print('Avg. loss by run')
for run_id in run_ids:
    print(f"{run_id}: {df[run_id+'_loss'].mean()}")

Avg. loss by run
oof_blurr_cocolm_large_2022_11_28_14_58: 0.37461858342313553
oof_blurr_deberta_v3_base_2022_11_23_16_10: 0.38311845953087975


## Optimize for loss

Prototype for how we can apply weights

In [ ]:
run_weights = np.array(np.ones(len(run_ids)))

for label in ["pred_cohesion","pred_syntax","pred_vocabulary","pred_phraseology","pred_grammar","pred_conventions"]:
    preds = []
    for idx, run_id in enumerate(run_ids):
        preds.append(df[f"{label}_{run_id}"].values * run_weights[idx])

    df[label] = np.stack(preds).sum(axis=0) / run_weights.sum()

df[f"weighted_loss"] = df.apply(calc_example_metric, axis=1)

    

In [ ]:
df[f"weighted_loss"].mean()

0.456843372710678

In [ ]:
df.head(10)

,id,fold,pred_cohesion_oof_blurr_cocolm_large_2022_11_28_14_58,pred_syntax_oof_blurr_cocolm_large_2022_11_28_14_58,pred_vocabulary_oof_blurr_cocolm_large_2022_11_28_14_58,pred_phraseology_oof_blurr_cocolm_large_2022_11_28_14_58,pred_grammar_oof_blurr_cocolm_large_2022_11_28_14_58,pred_conventions_oof_blurr_cocolm_large_2022_11_28_14_58,targ_cohesion,targ_syntax,...,pred_grammar_oof_blurr_deberta_v3_base_2022_11_23_16_10,pred_conventions_oof_blurr_deberta_v3_base_2022_11_23_16_10,oof_blurr_deberta_v3_base_2022_11_23_16_10_loss,pred_cohesion,pred_syntax,pred_vocabulary,pred_phraseology,pred_grammar,pred_conventions,weighted_loss
0,fold_0_0,0,3.162109,3.275391,3.298828,3.296875,3.458984,3.083984,2.5,2.5,...,1.785156,1.735352,0.312837,2.443359,2.432129,2.653320,2.546875,2.622070,2.409668,0.205084
1,fold_0_1,0,3.542969,3.664062,3.634766,3.691406,3.724609,3.429688,3.0,3.0,...,2.923828,2.882812,0.449223,3.234375,3.174805,3.354492,3.326172,3.324219,3.156250,0.236982
2,fold_0_2,0,3.638672,3.730469,3.662109,3.708984,3.638672,3.718750,4.5,4.0,...,3.929688,3.835938,0.301761,3.682617,3.687500,3.765625,3.817383,3.784180,3.777344,0.425620
3,fold_0_3,0,3.560547,3.529297,3.509766,3.380859,3.316406,3.742188,2.0,3.0,...,2.994141,3.228516,0.385096,3.519531,3.368164,3.468750,3.363281,3.155273,3.485352,0.976726
4,fold_0_4,0,3.306641,3.371094,3.410156,3.318359,3.332031,3.419922,4.0,4.0,...,2.968750,3.304688,0.440434,3.312500,3.199219,3.356445,3.238281,3.150391,3.362305,0.446942
5,fold_0_5,0,2.839844,2.775391,2.990234,2.757812,2.839844,2.908203,3.0,2.5,...,2.958984,3.390625,0.331384,3.171875,2.977539,3.215820,3.018555,2.899414,3.149414,0.272301
6,fold_0_6,0,2.673828,2.494141,2.884766,2.597656,2.544922,2.593750,2.5,2.5,...,2.808594,3.339844,0.260094,3.098633,2.841797,3.152344,2.916992,2.676758,2.966797,0.469891
7,fold_0_7,0,2.775391,2.666016,2.943359,2.742188,2.771484,2.650391,2.0,2.5,...,3.529297,3.490234,0.408859,3.134766,3.004883,3.239258,3.151367,3.150391,3.070312,0.462079
8,fold_0_8,0,2.880859,2.738281,2.976562,2.794922,2.783203,2.625000,4.0,3.0,...,2.916016,2.697266,0.470055,2.870117,2.702148,3.012695,2.889648,2.849609,2.661133,0.340015
9,fold_0_9,0,2.966797,2.871094,3.125000,2.890625,2.783203,2.953125,3.5,3.5,...,2.777344,2.630859,0.299488,2.895508,2.735352,3.069336,2.895508,2.780273,2.791992,0.661786


Tuning

In [ ]:
class OptimizeMetric:
    def __init__(self, run_ids) -> None:
        self.coefs_ = 0
        self.run_ids = run_ids
        self.n_models = len(self.run_ids)

    def _optimize(self, coefs, train_df, valid_df):
        for label in ["pred_cohesion","pred_syntax","pred_vocabulary","pred_phraseology","pred_grammar","pred_conventions"]:
            preds = []
            for idx, run_id in enumerate(self.run_ids):
                preds.append(valid_df[f"{label}_{run_id}"].values * coefs[idx])

            valid_df[label] = np.stack(preds).sum(axis=0) / coefs.sum()
        res = valid_df.apply(calc_example_metric, axis=1).mean()
        # print(res)
        return res

    def fit(self, train_df, valid_df):
        optimize_func = partial(self._optimize, train_df=train_df.copy(), valid_df=valid_df.copy())
        init_coefs = np.random.dirichlet(np.ones(self.n_models))
        optimize_res = fmin(optimize_func, init_coefs, disp=True)
        # optimize_res = minimize(optimize_func, init_coefs, method='Nelder-Mead', options={'maxiter': 10000, 'maxfev': 8000})
        self.coefs_ = optimize_res
    

In [ ]:
def tune_model_coefs(res_df, fold):
    train_df = res_df[res_df["fold"] != fold].reset_index(drop=True)
    valid_df = res_df[res_df["fold"] == fold].reset_index(drop=True)

    opt_metric = OptimizeMetric(run_ids)
    opt_metric.fit(train_df, valid_df)
    return opt_metric.coefs_

In [ ]:
fold_coefs = []
for f_idx in range(5):
    coefs = tune_model_coefs(df, f_idx)
    fold_coefs.append(coefs)

    print(f"Fold {f_idx}: {coefs}")

Optimization terminated successfully.
         Current function value: 0.374337
         Iterations: 37
         Function evaluations: 82
Fold 0: [0.61446169 0.0150783 ]


/tmp/ipykernel_1394859/3230640729.py:21: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  optimize_res = fmin(optimize_func, init_coefs, disp=True)


Fold 1: [0.17744436 0.82255564]
Fold 2: [0.25141205 0.74858795]
Fold 3: [0.80100426 0.19899574]
Fold 4: [0.28699177 0.71300823]


In [ ]:
fold_coefs

[array([0.61446169, 0.0150783 ]),
 array([0.17744436, 0.82255564]),
 array([0.25141205, 0.74858795]),
 array([0.80100426, 0.19899574]),
 array([0.28699177, 0.71300823])]

In [ ]:
np.array(fold_coefs).T

array([[0.61446169, 0.17744436, 0.25141205, 0.80100426, 0.28699177],
       [0.0150783 , 0.82255564, 0.74858795, 0.19899574, 0.71300823]])

In [ ]:
run_ids

{'oof_blurr_cocolm_large_2022_11_28_14_58',
 'oof_blurr_deberta_v3_base_2022_11_23_16_10'}

In [ ]:
preds_df = pd.DataFrame()

for f_idx, coefs in enumerate(fold_coefs):
    oof_df = df[df["fold"] == f_idx].reset_index(drop=True)
    
    for label in ["pred_cohesion","pred_syntax","pred_vocabulary","pred_phraseology","pred_grammar","pred_conventions"]:
        preds = []
        for idx, run_id in enumerate(run_ids):
            preds.append(oof_df[f"{label}_{run_id}"].values * coefs[idx])

        oof_df[label] = np.stack(preds).sum(axis=0) / coefs.sum()

    oof_df[f"weighted_loss"] = df.apply(calc_example_metric, axis=1)
    preds_df = pd.concat([preds_df, oof_df])

In [ ]:
print(len(preds_df))
preds_df.head(10)

3910


,id,fold,pred_cohesion_oof_blurr_cocolm_large_2022_11_28_14_58,pred_syntax_oof_blurr_cocolm_large_2022_11_28_14_58,pred_vocabulary_oof_blurr_cocolm_large_2022_11_28_14_58,pred_phraseology_oof_blurr_cocolm_large_2022_11_28_14_58,pred_grammar_oof_blurr_cocolm_large_2022_11_28_14_58,pred_conventions_oof_blurr_cocolm_large_2022_11_28_14_58,targ_cohesion,targ_syntax,...,pred_grammar_oof_blurr_deberta_v3_base_2022_11_23_16_10,pred_conventions_oof_blurr_deberta_v3_base_2022_11_23_16_10,oof_blurr_deberta_v3_base_2022_11_23_16_10_loss,pred_cohesion,pred_syntax,pred_vocabulary,pred_phraseology,pred_grammar,pred_conventions,weighted_loss
0,fold_0_0,0.0,3.162109,3.275391,3.298828,3.296875,3.458984,3.083984,2.5,2.5,...,1.785156,1.735352,0.312837,3.127679,3.234996,3.267907,3.260948,3.418894,3.051683,0.205084
1,fold_0_1,0.0,3.542969,3.664062,3.634766,3.691406,3.724609,3.429688,3.0,3.0,...,2.923828,2.882812,0.449223,3.528186,3.640626,3.621340,3.673911,3.705430,3.416589,0.236982
2,fold_0_2,0.0,3.638672,3.730469,3.662109,3.708984,3.638672,3.718750,4.5,4.0,...,3.929688,3.835938,0.301761,3.640777,3.728410,3.667068,3.714177,3.645642,3.721557,0.425620
3,fold_0_3,0.0,3.560547,3.529297,3.509766,3.380859,3.316406,3.742188,2.0,3.0,...,2.994141,3.228516,0.385096,3.558582,3.521578,3.507801,3.380017,3.308688,3.729884,0.976726
4,fold_0_4,0.0,3.306641,3.371094,3.410156,3.318359,3.332031,3.419922,4.0,4.0,...,2.968750,3.304688,0.440434,3.306921,3.362861,3.407583,3.314523,3.323330,3.417162,0.446942
5,fold_0_5,0.0,2.839844,2.775391,2.990234,2.757812,2.839844,2.908203,3.0,2.5,...,2.958984,3.390625,0.331384,2.855749,2.785074,3.001041,2.770303,2.842697,2.919758,0.272301
6,fold_0_6,0.0,2.673828,2.494141,2.884766,2.597656,2.544922,2.593750,2.5,2.5,...,2.808594,3.339844,0.260094,2.694177,2.510794,2.897583,2.612953,2.551237,2.611620,0.469891
7,fold_0_7,0.0,2.775391,2.666016,2.943359,2.742188,2.771484,2.650391,2.0,2.5,...,3.529297,3.490234,0.408859,2.792606,2.682248,2.957534,2.761788,2.789635,2.670506,0.462079
8,fold_0_8,0.0,2.880859,2.738281,2.976562,2.794922,2.783203,2.625000,4.0,3.0,...,2.916016,2.697266,0.470055,2.880345,2.736550,2.978293,2.799460,2.786384,2.626731,0.340015
9,fold_0_9,0.0,2.966797,2.871094,3.125000,2.890625,2.783203,2.953125,3.5,3.5,...,2.777344,2.630859,0.299488,2.963382,2.864591,3.122334,2.890859,2.783063,2.945406,0.661786


In [ ]:
print("Avg. weighted loss")
preds_df.weighted_loss.mean()

Avg. weighted loss


0.456843372710678

## Export -

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
